<a href="https://colab.research.google.com/github/m-fila/uczenie-maszynowe-2021-22/blob/main/03_Regresja_logistyczna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresja logistyczna, walidacja krzyżowa, krzywe ROC
Autor: Jarosław Żygierewicz

## Część I: regresja logistyczna
Ten notebook pomoże Ci zapoznać się z regresją logistyczną. 

Zbudujemy klasyfikator bazujący na regresji logistycznej. Jego zadaniem będzie określanie prawdopodobieństwa przyjęcia kandydata na studia na podstawie wyników z dwóch egzaminów maturalnych (każdy przeskalowany na zakres 0-100%): z matematyki i z biologii. 

Zanim przejdziemy do właściwych zadań zaimportujmy potrzebne moduły:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so

Zestaw uczący pobieramy z repozytorium github i wczytujemy używając klasy Pandas DataFrame. Najpierw pobieramy repozytorium z github. Repozytorium zawiera kod do naszych ćwiczeń, oraz przykładowe dane w katalogu "dane".

In [ ]:
# odkomentuj zeby pobrac repozytorium, mozesz tez wgrac samemu odpowiedni plik z danymi
#!git clone https://github.com/m-fila/uczenie-maszynowe-2021-22

In [ ]:
df = pd.read_csv("uczenie-maszynowe-2021-22/dane/reg_log_data.txt", encoding='latin-1', sep=",", names=["matematyka", "biologia", "wynik"])

Aby łatwiej było się nimi posługiwać wydzielmy z nich dane wejściowe jako 'X' i wyjściowe jako 'Y':

## Analiza wizualna danych. 

Pierwszy krok przy analizie danych z użyciem dowolnego algorytmu to ich inspekcja. Korzystając z metod klasy DataFrame proszę:
* wypisać na ekran framgent danych
* narysować rozkłady wszystkich zmiennych wejściowych, w naszym przypadku wyniku egzaminów z matematyki i biologii dla całego zbioru
* narysować rozkłady wszystkich zmiennych wejściowych, w naszym przypadku wyniku egzaminów z matematyki i biologii dla wierszy gdzie wynik=0
* narysować rozkłady wszystkich zmiennych wejściowych, w naszym przypadku wyniku egzaminów z matematyki i biologii dla wierszy gdzie wynik=1

**Wskazówka**: proszę użyć filtrowania danych, tak jak to było robione na pierwszych zajęciach.

In [ ]:
# uzyj tego do okreslenia rozmiaru obrazka
figsize=(15,5)
# wypisz dataframe
...
# narysuj histogramy dla wszystkich kolumn
...
# wyplotuj tych co zdali
...
# wyplotuj tych co nie zdali
...

#To samo co powyżej w jednej linii
df.hist(figsize=figsize, by="wynik");

To co nas interesuje to są jednak korelacje między zmiennymi. Korzystając z przykładu z pierwszych ćwiczeń proszę:

* narysować wykres korelacji między zmiennymi wejściowymi dla pełnych danych, oraz wierszy gdzie wynik=0 lub 1

**Wskazówka**: proszę użyć parametru "hue" funkcji sns.jointplot()

In [ ]:
import seaborn as sns

x = sns.jointplot(...)
x.set_axis_labels('matematyka', 'biologia', fontsize=16);

x = sns.jointplot(...)
x.set_axis_labels('matematyka', 'biologia', fontsize=16);

x = sns.jointplot(...)
x.set_axis_labels('matematyka', 'biologia', fontsize=16);

## Hipoteza
Dla przypomnienia _hipoteza_ w regresji logistycznej ma postać: 

$\qquad$ $h_\theta(x) = \frac{1}{1+\exp(-\theta x^T )}$.

W implementacji dobrze jest myśleć o tej funkcji tak:

$\qquad$ $h_\theta(x) = \frac{1}{1+f}$.

gdzie: $f = \exp(-\theta x^T)$

Proszę napisać funkcję ```logistic_func(x, theta)``` która:

* implementuje funkcję logistyczną
* jako argumenty przyjmuje parametry regresji logistycznej  $(\theta_{0}, \theta_{1}, ..., \theta_{i})$ oraz tablicę danych wejściowych $x$. 
* w kodzie fukcji proszę rozszerzyć tablicę $x$ o dodatkową kolumnę jedynek, by parametr $\theta_{0}$ był traktowany na tej same zasadzie co pozostałe parametry
* ze względu na stabilność numeryczną obliczeń ma ograniczony zakres zmienności. Proszę ograniczyć wartości wykładnika w mianowniku do zakresu  $\pm18$

**Ostrzeżenie:** x to tablica która może zawierać wiele kolumn i wiele wierszy.

**Wskazówka**: ograniczając zakres zwracanych wartości proszę skorzystać z funkcji np.where() zaaplikowanej do wektora wartości wykładnika.

Proszę sprawdzić działanie funkcji na następujących danych testowych:
```
theta = np.array([1,1,2])
x = np.array([[5,5],
              [5,6],
              [-5,-5],
              [-5,-8]])
```

In [ ]:
def logistic_func(theta, x):
    # dodaj kolumne jedynek
    x_expanded = ...
    # policz argument funkcji
    arg = ...
    # uzyj np.where żeby ograniczyc wartosci parmetru do [-18,18]
    arg = ...
    return 1.0/(1+np.exp(-arg))



theta = np.array([1,1,2])
x = np.array([[5,5],
              [5,6],
              [-5,-5],
              [-5,-8]])
res = logistic_func(theta, x)
# poprosze liste o podanym wymiarze na wyniku
assert res.shape == (4,)
print(res)


## Funkcja log-wiarygodności: 
Parametry regresji znajdujemy przez maksymalizację [funkcji log-wiarygodności](https://brain.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wykład_6#Funkcja_wiarygodno.C5.9Bci):

$\qquad$ $l(\theta) = \log L(\theta) = \sum_{j=1}^m y^{(j)} \log h(x^{(j)}) + (1 - y^{(j)}) \log (1 - h(x^{(j)}))$,
gdzie:  

m - liczebność próbki

x - dane wejściowe, u nas wyniki z egaminów z matematyki i biologii

y - dane wyjściowe, u nas wynik rekrutacji na studia

h - postać zależności wyniku od danych wejściowych. U nas to jest funkcja logistyczna, czyli oczekujemy, że wzór y = h(x) dobrze opisuje zależnośc między danymi wejściowymi, a wyjściowymi.


<hr>

Proszę napisać funkcję ```log_likelihood(theta, x,y, model)``` która:

* implementuje funkcję log-wiarygodności
* jako argumenty przyjmuje parametry regresji logistycznej  $(\theta_{0}, \theta_{1}, ..., \theta_{i})$ oraz tablicę danych wejściowych $x, y$. 
* model dla którego szukamy parametrów $\theta_{i}$ w naszym przypadku to będzie funkcja logistyczna: ```logistic_func```

**Uwaga**: argument $theta$ musi być pierwszy 

In [ ]:
def log_likelihood(theta, x, y, model):
    # Miejsce na twój kod. Użyj argumentu model zamiast konkretnej funkcji.
    ...

Maksymalizacja to zadanie optymalizacyjne - szukamy optymalnych parametrów, a kryterium optymalności to maksymalna wartość funckji log-wiarygodności.
W tym ćwiczeniu zrobimy to za pomocą funkcji optymalizacyjnych z modułu [<tt>scipy.optimize</tt>]( http://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize). 


Wynikają z tego dwie konsekwencje:
* funkcje te są przystosowane do szukania minimów funkcji celu. Musimy więc podawać im jako argumenty funkcję minus log-wiarygodności
* niektóre algorytmy mogą działać szybciej jeśli zaimplementujemy jawnie postać pochodnej:

$\qquad$ $
\begin{array}{lcl}
\frac{\partial}{\partial \theta_i} l(\theta)  =\sum_{j=1}^m (y^{(j)}-h_\theta(x^{(j)}))x_i^{(j)}
\end{array}
$

Proszę napisać funkcję ```negative_log_likelihood(theta, x,y, model)``` która:

* zwraca funkcję log-wiarygodności pomnożoną przez $-1$

In [ ]:
def negative_log_likelihood(theta, x, y, model):
    ...

Proszę napisać funkcję ```log_likelihood_derivative(theta, x,y, model)``` oraz ```negative_log_likelihood_derivative(theta, x,y, model)``` które:

* zwraca funkcję pochodną log-wiarygodności
* zwraca funkcję pochodną log-wiarygodności pomnożoną przez $-1$

**Uwaga**: mnożąc przez $x_{i}$ trzeba uwzględnić kolumnę jedynek

In [ ]:
def log_likelihood_derivative(theta, x, y, model):
    # Miejsce na Twój kod. Pamiętaj, żeby dodać kolumnę jedynek do x.
    #
    # 1. Policz wynik modelu dzialajacego na danych wejsciowych
    ...
    # 2. Policz różnicę względem danych wyjściowcyh
    ...
    # 3. Policz wartość pochodnej pamiętając o sumowaniu
    ...
    # 4. zwróc wynik, wartość pochodnej we wszystkich kierunkach (kolumnach)
    assert result.shape == theta.shape
    return result

def negative_log_likelihood_derivative(theta, x, y, model):
    return -log_likelihood_derivative(theta, x, y, model)

## Procedura minimalizacji funkcji log-wiarygodności ze wsględu na parametry $\theta$ dla konkretnych danych.

W naszym przypadku mamy trzy parametry $\theta$ - mnożące odpowiednio 1, wynik z matematyki, wynik z biologii.

Proszę:
* zainicjalizować parametry $\theta_{0}, \theta_{1}, \theta_{2}$ na wartości 0.
* obliczyć wartość i pochodną funkcji wiarygodności na danych początkowych 

Poprawne wartości to:
```
Wartość funkcji log-wiarygodności dla zbioru testowego = -69.31471805599453
Wartość pochodnej funkcji log-wiarygodności dla zbioru testowego = [  10.         1200.92165893 1126.28422055]
```

In [ ]:
theta0 = ...
model = ...

# wartość funkcji log-wiarygodności
llh = ...
# wartość pochodnej
llh_derivative = ...

print("Wartość funkcji log-wiarygodności dla zbioru testowego = {}".format(llh))
print("Wartość pochodnej funkcji log-wiarygodności dla zbioru testowego = {}".format(llh_derivative))

## Optymalizacja  

Funkcje optymalizujące zaczerpniemy z modułu scipy.optimize: ```scipy.optimize.fmin_bfgs```. Ponieważ funkcje te są zaimplementowane do mnimalizowania to zamiast maksymalizować funkcję low-wiarygodności będziemy minimalizować tą funkcje przemnożoną przez -1 czyli ```f=negative_log_likelihood``` oraz ```fprime=negative_log_likelihood_derivative```


* proszę wywołać funckję ```scipy.optimize.fmin_bfgs``` z obpowiednimi argumentami.
* proszę porównać liczbę wywołań i czas wykonywania komórki z i bez podania explicite postaci pochodnej
(https://ipython.readthedocs.io/en/stable/interactive/magics.html?highlight=%25time#cell-magics)

In [ ]:
%%time
# ^ rozpoczecie mierzenia czasu dzieje sie tu


model = ...

# znajdz optymalne parametry theta
theta_opt = so.fmin_bfgs(...)
# policz log-wiarygodnosc z optymalnymi parametrami
llh = log_likelihood(...)

print('Optymalne wartości parametrów theta: {}'.format(theta_opt))
print("Wartość funkcji log-wiarygodności dla optymalnych parametrów: {}".format(llh))

## Wyniki
Wyniki regresji logistycznej możemy odbierać na dwa sposoby:
* obliczyć wartość hipotezy dla badanego wejścia i dopasowanych parametrów: miara ta ma interpretację prawdopodobieństwa przynależności wejścia do klasy 1,
* dopisać funkcję wykonującą klasyfikację, tzn. porównanie wartości hipotezy z 1/2: 
  * dla wartości hipotezy > 1/2 klasyfikacja zwraca 1, 
  * w przeciwnym razie 0.
  
  
Proszę napisać funkcję ```classification(theta, x)```  która:
* jako argument przyjmuje wektora parametrów modelu $\theta$, tablicę danych wejściowych $x$, oraz $model$
* zwraca listę klasyfikacyjną: $1$ gdy $model(x)>0.5$, a $0$ w przeciwnym przypadku

In [ ]:
def classification(theta, x, model):
    ...

## Przewidywanie 

Proszę:

* korzystając z modelu ```logistic_func``` wraz z parametrami zwróconymi przez procedurę optymalizacyjną obliczyć prawdopobobieństwo zdania
  osoby, która uzyskała 20 pkt. z matematyki, oraz 80 z biologii.
* korzystając z funckji "classification" wyznaczyć czy osoba należy do   klasy $0$ czy $1$.

In [ ]:
# stworz dane odpowiadajace naszej osobie
x = ...
# policz prawdopodobienstwo zdania
p = ...

print("Osoba z {} pkt z matematyki, oraz {} pkt. z biologii ma {}% szans na przyjęcie na studia.".format(x[0,0], x[0,1], round(p[0]*100,3)))

# sklasyfikuj osobę
class_number = ...
print("Osoba zalicza się do klasy: {}".format(class_number))

Narysujmy uzyskany podział. Na tle punktów pokolorowanych zgodnie z przynależnością do klas dorysujemy prostą rozgraniczającą obszary "1" od "0".   Ma ona równanie 

$\qquad$ $h_\theta(x)=1/2$, 

tzn:

$\qquad$ $\theta x^T = 0$

czyli 

$\theta_0 +\theta_1 x_1 + \theta_2 x_2 =0 $

Przekształcając to do równania prostej we współrzędnych $(x_1,x_2)$ mamy:

$- \theta_2 x_2 = \theta_0 +\theta_1 x_1 $

$ x_2 = - \frac{1}{\theta_2}( \theta_0 +\theta_1 x_1 )$

In [ ]:
df_passed = df[df["wynik"]==1]
df_failed = df[df["wynik"]==0]

fig, axes = plt.subplots(1,1, figsize=(5,5))
# narysuj tych co zdali na niebiesko, pamietaj o argumencie "label" dla czytelnosci
...
# narysuj tych co nie zdali na czerwono, pamietaj o argumencie "label" dla czytelnosci
...

# znajdz prostą
x = ...
y = ...

# rysowanie prostej i legendy
axes.plot(x,y, label="logistic regression model")
axes.legend();

## Część II: Walidacja - to na ćwiczenia w przyszłym tygodniu
Teoria do tej części znajduje się tu:

https://brain.fuw.edu.pl/edu/index.php/Uczenie_maszynowe_i_sztuczne_sieci_neuronowe/Wykład_Ocena_jakości_klasyfikacji

### Zastosowanie w naszym przykładzie
Dodamy teraz kross-walidację typu $leave-one-out$.
Po kolei odłożymy po jednym przykładzie ze zbioru uczącego i na takim zredukownaym zbiorze nauczymy regresję, a następnie sprawdzimy 
działanie modelu na odłożonym przykładzie:

<ol>
<li> ze zbioru uczącego odrzucamy jeden przykład </li>
<li> na pozostałych przykładach "trenujemy model", czyli znajdujemy parametry $\theta$ </li>
<li> sprawdzamy działanie modelu na odrzuconym wcześniej przykładzie</li>
<li> procedurę powtarzamy dla wszystkich przykładów w zbiorze uczącym </li>   
</ol> 

Proszę napisać funckję ```leave_one_out_CV(df, theta, model)```

która:
* przyjmuje zestaw uczący w postaci obiektu DataFrame, początkowych parametrów $\theta$, oraz model $model$
* wykonuje operację "leave-one-out" i tworzy listę wyników modelu dla każdego przykładu:

```
passed = np.append(passed, classification(theta_opt, df_left_out[["matematyka","biologia"]], model))
```

* dodaje do obiektu DataFrame kolumnę z wynikami modelu:

```
df["model"] = passed 
```   

In [ ]:
%%time 

def leave_one_out_CV(df, theta0, model):
    # tutaj bedziemy wpisywac wyniki modelu
    passed = np.array([])
    # tworzymy kopie data frame
    df_with_model = df.copy()
    
    # pętla po wszystkich przykładach
    for leave_out_index in df.index:
        # 1. stworz dataframe bez jednego przykladu
        df_filtered = ...
        # 2. znajdz optymalne parametry theta
        theta_opt = ...
        # 3. stworz dataframe z odrzuconego (pojedynczego) przykladu
        df_left_out = ...
        # 4. dodajemy wynik modelu do poprzednich
        passed = ...
    # Dodajemy wyniki modelu do df_with_model
    ...
    # zwracamy data frame powiekszony o kolumne z wynikami modelu
    return df_with_model
                        
theta0 = np.array([0,0,0])
model = logistic_func 
df_with_model = leave_one_out_CV(df, theta0, model)
print(df_with_model)

Zdefiniujmy następujące przypadki gdy nasz model się myli lub podaje poprawny wynik:

* "True Positive" (TP):  stan faktyczny jest pozytywny (y=1) i klasyfikator się nie myli (wynik = 1)
* "True Negative" (TN):  stan faktyczny jest negatywny (y=0) i klasyfikator się nie myli (wynik = 0) 
* "False Positive" (FP): wynik fałszywie pozytywny (fałszywy alarm): stan faktyczny jest negatywny (y=0) ale klasyfikator się  myli (wynik = 1)
* "False Netative" (FN): przegapiony alarm: stan faktyczny jest pozytywny (y=1) i klasyfikator się myli (wynik = 0)

Proszę napisać kod, który oblicza TP, TN, FP, FN. Dla naszego zbioru uczącego powinniśmy uzyskać:
```
TP:  55
FP:  6
TN:  34
FN:  5
```   

**Wskazówka:** proszę zliczać liczbę wierszy w odpowiednio przefiltrowanym obiekcie DataFrame

In [ ]:
# podpowiedz: uzyj warunkow logicznych aby wybrac odpowiednie dane i pola shape aby dostac liczbe wierszy
tp = ...
fp = ...
tn = ...
fn = ...

print("TP = {}\nFP = {}\nTN = {}\nFN = {}".format(tp, fp, tn, fn))

## Krzywa ROC

Aby wykreślić krzywą ROC należy przeprowadzić klasyfikację dla wielu możliwych wartości progu dla hipotezy, powyżej którego uznajemy przypadek za należący do klasy 1. W tym celu należy zmodyfikować funckję ```leave_one_out_CV(df, theta0, model)``` tak by zapisywała prawdopodobieństwo, a nie wynik działania funkcji ```classification```

Proszę napisać funkcję ```leave_one_out_CV_with_prob(df, theta0, model)``` 
* która zapisuje kolumnę z prawdopodobieństwem zamiast wynikiem klasyfikacji

In [ ]:
%%time 

def leave_one_out_CV_with_prob(df, theta0, model):
    # tutaj bedziemy wpisywac wyniki modelu (prawdopodobienstwa)
    prob = np.array([])
    # robimy kopie oryginalnych danych do ktorych dodamy kolumne
    df_with_model = df.copy()
    # petla po wszystkich przykladach
    for leave_out_index in df.index:
        # 1. stworz dataframe bez jednego przykladu
        ...
        # 2. znajdz optymalne parametry theta
        ...
        # 3. stworz dataframe z odrzuconego (pojedynczego) przykladu
        ...
        # 4. dodajemy wynik modelu do poprzednich
        ...
    # Dodajemy wyniki modelu (prawdopodobienstwa) do calego data frame
    ...
    # zwracamy data frame powiekszony o kolumne z wynikami modelu (prawdopodobienstwami)
    return df_with_model
                        
theta0 = np.array([0,0,0])
model = logistic_func 
df_with_prob = leave_one_out_CV_with_prob(df, theta0, model)
print(df_with_prob)

Korzystając z biblioteki ```sklearn``` proszę narysować krzywą ROC oraz obliczyć pole pod nią ("area unde ROC, AUC) dla naszego modelu.

**Wskazówka:** wpisać w Google hasło "scikit learn Receiver Operating Characteristic"

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

fpr, tpr, thresholds = ...
roc_auc = ...

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
# narysujmy krzywą (diagonala) dla rzutu monetą (naiwny klasyfikator, najgorsze rozwiązanie)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for logistic regression')
plt.legend(loc="lower right")
plt.show()

## Modyfikacja modelu

Proszę wykonać trening regresji logistycznej dla modelu, który używa wyniku tylko z jednego egzaminu i narysować na jednym rysunku krzywe ROC dla trzech wariantów:
* modelu używającego wyników z obu przedmiotów
* modelu używającego tylko wyników z matematyki
* modelu używającego tylko wyników z biologii

**Wskazówka**: należy przerobić funkcję ```leave_one_out_CV_with_prob``` tak by wykonywała obliczenia dla wszystkich trzech wariantów

In [ ]:
%%time

def leave_one_out_CV_many_models(df, theta0, model):
    # tu zapisujemy wszystkie prawdopodobienstwa
    prob = np.array([])
    # tu tylko dla matematyki
    prob_math = np.array([])
    # tu tylko dla biologii
    prob_biol = np.array([])
    # tu robimy kopie
    df_with_model = df.copy()
    
    for leave_out_index in df.index:
        # robimy podobnie jak poprzednio, ale tym razem 3 razy: dla obu dziedzin, tylko dla majcy, tylko dla biologii
        ...
        ...
        ...
        ##########
    # dodajemy kolumny do data frame i zwracamy
    ...
    ...
    ...
    return df_with_model
                        
theta0 = np.array([0,0,0])
model = logistic_func 
df_with_prob = leave_one_out_CV_many_models(df, theta0, model)
print(df_with_prob)

In [ ]:
# analogicznie jak poprzednio, ale tym razem liczymy i plotujemy 3 ROC 
fpr, tpr, thresholds = ...
fpr_math, tpr_math, thresholds = ...
fpr_biol, tpr_biol, thresholds = ...

roc_auc = ...
roc_auc_math = ...
roc_auc_biol = ...

plt.figure()
lw = 2
# uzupelnij kod poniżej
...
...
...
#
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for logistic regression')
plt.legend(loc="lower right")
plt.show()

## Zastosowanie do innego rodzaju danych

Proszę przeprowadzić procedurę treningu i narysować krzywą ROC dla danych gdzie występuje inny podział między klasami:


In [ ]:
nPoints = 100
x = 100*np.random.random_sample(nPoints)
y = 100*np.random.random_sample(nPoints)

df = pd.DataFrame(data=x, columns=["matematyka"])
df["biologia"] = y
# tworzymy nowe dane
df["wynik"] = np.sqrt((x-50)**2 + (y-50)**2)<25
# narysuj dwuwymiarowy wykres aby zobaczyc korelacje
x = ...
x.set_axis_labels('matematyka', 'biologia', fontsize=16);

In [ ]:
theta0 = np.array([0,0,0])
model = logistic_func
# policz prawdopodobienstwa z modelu przy uzyciu walidacji leave_one_out
df_with_prob = ...
print(df_with_prob)

In [ ]:
# narysuj ŁADNY wykres z krzywą ROC i wypisz AUC w legendzie
...

## Co można zrobić by poprawić działanie modelu na takich danych?

Napisz funkcję logistic_func_1(theta, x) która będzie działać podobnie do oryginalnej, ale dodaj w niej do oryginalnych danych 3 kolumny: kolumnę jedynek (tak jak poprzednio), kolumnę x1^2, kolumnę x2^2, gdzie x1 i x2 to wyniki z matematyki i biologii odpowiednio.

Następnie uzupełnij funkcję leave_one_out_CV_with_prob(df, theta0, model), analogicznie do poprzednich funkcji. Funkcja ma wykonać walidację leave-one-out i zwrocić prawdopodbieństwo uzyskane z modelu.

In [ ]:
%%time

def logistic_func_1(theta, x):
    ...

def leave_one_out_CV_with_prob(df, theta0, model):
    
    prob = np.array([])
    df_with_model = df.copy()
    # napisz petle po przypadkach analoficznie jak poprzednio
    ...
    # dodaj prob jako nową kolumnę
    ...
    return df_with_model

theta0 = np.array([0,0,0,0,0])
model = logistic_func_1 
df_with_prob = leave_one_out_CV_with_prob(df, theta0, model)
print(df_with_prob)

In [ ]:
# narysuj ładny wykres ROC tak jak poprzednio ale dla nowego modelu
...